In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
# read the data
filepath_train = "/Users/tungvuduc/Desktop/PhD/projects/repos/notebooks/notebooks/assets/dna/datasets/CTCF_train_sequences.csv"
filepath_test = "/Users/tungvuduc/Desktop/PhD/projects/repos/notebooks/notebooks/assets/dna/datasets/CTCF_test_sequences.csv"
df_train = pd.read_csv(filepath_train)
df_test = pd.read_csv(filepath_test)

df_test

,sequence,label,transcription_factor,subset
0,AAGCCTCACTTTGCGCTGTCCCTGCTCTGAATATCTGTCATGCCTT...,0,CTCF,test
1,GAGGCGGCTCGGGTGAGCTGTCAGCCCATGCACAAGTGACTTTTTA...,0,CTCF,test
2,GCAGAAGTCCGCGGACGAGAAGAGCCGCAGCACGTTGACCAGAGGG...,1,CTCF,test
3,TGCTCAGTTCAGCTCAGCTCAGGCCAGTCCAGCCCAGCTCAGGCCA...,1,CTCF,test
4,TCTTTATCCTTCATTTCAATACAGAAGATGCTTCATATGCACAGTG...,0,CTCF,test
...,...,...,...,...
17448,CAATCATGGATCATAATGGATTCCATGGTGAACTGAACAGGGTGCT...,1,CTCF,test
17449,TCGCCCTTCATATGCCTGTGCTGCGTGCTTCGAGGACTTCCCTCTG...,1,CTCF,test
17450,GCCGCTCGGTGTCGGGTCTCTGTGCCTGAGCGAGGACCCCAACCCT...,1,CTCF,test
17451,CCAGCAGCATGCGGACATGAAGGCCCAGCTCCCTCATCTCAACTGG...,1,CTCF,test


In [48]:
path_to_data = "/Users/tungvuduc/Desktop/PhD/projects/repos/tfbind/data"
files = os.listdir(path_to_data)
protein_names = [name.split("_")[0] for name in files]

def combine_train_val_test(path_to_data):
    files = os.listdir(path_to_data)
    train_files = [pd.read_csv(os.path.join(path_to_data, file)) for file in files if "train" in file]
    val_files = [pd.read_csv(os.path.join(path_to_data, file)) for file in files if "valid" in file]
    test_files = [pd.read_csv(os.path.join(path_to_data, file)) for file in files if "test" in file]

    train_files = [file.rename(columns={"label" : file["transcription_factor"][0]}).iloc[:, :2] for file in train_files]
    val_files = [file.rename(columns={"label": file["transcription_factor"][0]}).iloc[:, :2] for file in val_files]
    test_files = [file.rename(columns={"label": file["transcription_factor"][0]}).iloc[:, :2] for file in test_files]

    train_files_combined = reduce(lambda data1, data2: data1.merge(data2, on="sequence", how='outer'), train_files).fillna(0)
    val_files_combined = reduce(lambda data1, data2: data1.merge(data2, on="sequence", how='outer'), val_files).fillna(0)
    test_files_combined = reduce(lambda data1, data2: data1.merge(data2, on="sequence", how='outer'), test_files).fillna(0)
    
    return train_files_combined, val_files_combined, test_files_combined

train, val, test = combine_train_val_test(path_to_data=path_to_data)

In [54]:
# save the data
train.to_parquet(os.path.join(path_to_data, "train_sequences.parquet"))
val.to_parquet(os.path.join(path_to_data, "val_sequences.parquet"))
test.to_parquet(os.path.join(path_to_data, "test_sequences.parquet"))